## Import Packages

In [ ]:
import os
import plotly
import pandas as pd
import fbprophet as ph
from fbprophet.plot import plot_plotly, plot_components_plotly
from math import floor
from tqdm.notebook import tqdm

## Import Dataset
### Source
Hourly energy demand generation and weather  
https://www.kaggle.com/nicholasjhana/energy-consumption-generation-prices-and-weather
### Content

This dataset contains 4 years of electrical consumption, generation, pricing, and weather data for Spain. Consumption and generation data was retrieved from ENTSOE a public portal for Transmission Service Operator (TSO) data. Settlement prices were obtained from the Spanish TSO Red Electric España. Weather data was purchased as part of a personal project from the Open Weather API for the 5 largest cities in Spain and made public here.  
  
  
### Inspirational Problems to Solve
- What weather measurements, and cities influence most the electrical demand, prices, generation capacity?
- Forecast 24 hours in advance?
- Predict electrical price by time of day?
- Forecast intraday price or electrical demand hour-by-hour.

In [ ]:
energy_df = pd.read_csv('dataset/hourly_energy_demand_generation_and_weather/energy_dataset.csv')
weather_df = pd.read_csv('dataset/hourly_energy_demand_generation_and_weather/weather_features.csv')
print(energy_df.shape)
print(weather_df.shape)

In [ ]:
energy_df.columns

In [ ]:
energy_df.head()

In [ ]:
weather_df.head()

## Data Prep
Prophet requires time series data to have a minimum of two columns: `ds` which is the time stamp and `y` which is the values

In [ ]:
energy_df['time'] = pd.to_datetime(energy_df['time'], utc=True)
# Remove timezone
energy_df['time'] = energy_df['time'].dt.tz_localize(None)
energy_df.sort_values('time', inplace=True)

In [ ]:
len(energy_df)

In [ ]:
hist_size = 2/3
energy_hist = energy_df.head(floor(hist_size * len(energy_df)))[['time', 'price day ahead']]
energy_hist.columns = ['ds', 'y']
energy_hist.shape

### Predict with Prophet

In [ ]:
prophet = ph.Prophet()

In [ ]:
prophet.fit(df=energy_hist)

In [ ]:
# Add in 90 days in future
energy_future = prophet.make_future_dataframe(periods=365*2)

In [ ]:
energy_future.tail()

In [ ]:
forecast = prophet.predict(energy_future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = prophet.plot(forecast, figsize=(12,6))

### Trend, yearly seasonality, and weekly seasonality of the time series

In [ ]:
fig2 = prophet.plot_components(forecast)

In [ ]:
# plot_plotly(prophet, forecast)